##  **Voting methods**

In this practical, you will experiment several voting methods on real world data, namely 


* either on the 2002 Irish General Election (https://en.wikipedia.org/wiki/Dublin_North_(D%C3%A1il_constituency)#2002_general_election), on Dublin North data
* or on the 2017 French presidentail election, on the data provided by the city of Crolles and on the data provided in martinique (ecole dillon)  (see https://vote.imag.fr/about et https://zenodo.org/record/1199545/files/voter-autrement-online.pdf?download=1   and https://hal.univ-antilles.fr/hal-01965660/document pour plus de details)for more details)

(you can choose on work either on the Irish general election, or on the french presidential election or on both)

All the data files are on moodle

**French Election, 11 candidates :  file SVT111.cvs** contains the preference order of the voters -  each voter gives 11 to his/her preferred candidate, 10 to the second, etc. The candidate(s) not approved by the voter receive rate 0. Each line corresponds toa voter

**Irish Election, 12 candidates :  file Irish.cvs** contain the preference order of the voters -  each voter gives 12 to his/her preferred candidate, 11 to the second, etc.The candidate(s) not approved by the voter receive rate 0.  Notice that the first column of a line indicate the number of voters whixh expressed the ballot described by the line

For questions 1, 2 and 3 you can use an excel/calc sheet  




**Question 1: determine the winner according to Borda's rule**



*Add your result to the google sheet*  https://docs.google.com/spreadsheets/d/1ZkSVoET8AtS-BCUYmYPo1VQAQsTCOf0Xjtz_Xxd5-L8/edit?usp=sharing

**Question 2: : determine the winner according to Transferable voting  (IRV)**

Add your result to the google sheet https://docs.google.com/spreadsheets/d/1ZkSVoET8AtS-BCUYmYPo1VQAQsTCOf0Xjtz_Xxd5-L8/edit?usp=sharing





**Question 3: let us now explore condorcet coherent methods**

Summary data are provided by files Dillon.cvs   and IrishCondorcet.cvs


Determine whether there is a Condorcet winner and compute the order on the candidates given by Copeland's rule



*Add your result to the google sheet*  https://docs.google.com/spreadsheets/d/1ZkSVoET8AtS-BCUYmYPo1VQAQsTCOf0Xjtz_Xxd5-L8/edit?usp=sharing

##  **e-voting**  (dead line : 30th of november)

Implement a python toolbox which allows the computation, for any file  the format irish.cvs/SVT111.cvs of

    
    * The Condorcet winner (if any)
    * Copeland's preference order
    * Kramer simpson's preference order
    * Borda  preference order
    * STV  preference order
    * The winner elected by the plurality rule
    
    

Test your code on irish.cvs, SVT111.cvs  and SVT411.cvs 

**Your toolbox + demo will be provided under the form of a notebook, for november the 30th - on moodle**

    

### The Bordas Rule : 

In [55]:
## Bordas Rule

import pandas as pd
import numpy as np

df = pd.read_csv('Irish.csv', skiprows=range(4),encoding='latin1')

dfnew = df.copy()

for col in df.columns[1:]:
    dfnew[col] = df['number of voters']*df[col]
        
dfnew
df['number of voters'].sum()

candidates = df.columns[1:]
sumI = [dfnew[can].sum() for can in candidates]

sumI = np.array(sumI)

orderid = np.argsort(sumI)[::-1]

np.sort(sumI)[::-1]

[(candidates[i],sumI[i]) for i in orderid]


[('Trevor Sargent G.P. ', 261376.0),
 ('Sean Ryan Lab ', 223554.0),
 ('Jim Glennon F.F. ', 199348.0),
 ('Michael Kennedy F.F. ', 196552.0),
 ('G.V. Wright F.F. ', 186029.0),
 ('Clare Daly S.P. ', 171961.0),
 ('Nora Owen F.G. ', 146013.0),
 ('Cathal Boland F.G. ', 87491.0),
 ('Ciaran Goulding Non-P ', 61744.0),
 ('Mick Davis S.F. ', 56420.0),
 ('Eamonn Quinn Non-P ', 24170.0),
 ('David Henry Walshe C.C. Csp ', 13841.0)]

### The Condorcet winner: 

In [5]:
## The Condorcet winner (if any)

import pandas as pd
import numpy as np

def append_value(dict_obj, key, value):
    if key in dict_obj:
        if not isinstance(dict_obj[key], list):
            dict_obj[key] = [dict_obj[key]]
        dict_obj[key].append(value)
    else:
        dict_obj[key] = [value]


df = pd.read_csv('Irish.csv', skiprows=range(4),encoding='latin1')
dfResult = df.copy()
df = df.drop("number of voters",1)

finalData = {}

for i in df.columns:
    for j in dfResult.columns[1:]:
        if i != j:
            totalI = 0
            totalJ = 0
            #  comparing 2 columns
            totalI = sum(np.where(dfResult[i] >= dfResult[j], dfResult['number of voters'], 0))
            totalJ = sum(np.where(dfResult[i] <= dfResult[j], dfResult['number of voters'], 0))
            if totalI > totalJ:
                append_value(finalData, i, totalI)
            elif totalI < totalJ:
                append_value(finalData, j, totalJ) 
            else:
                append_value(finalData, i, totalI)
                append_value(finalData, j, totalJ)                
    dfResult = dfResult.drop(i, 1)


sorted_data = sorted(finalData, key=lambda k: len(finalData[k]), reverse=True)

print(sorted_data[0] + "is the Condorcet winner")



Trevor Sargent G.P. is the Condorcet winner


### The Kramer Simpsons Rule: 

In [29]:
## KS rule

import pandas as pd
import numpy as np

df = pd.read_csv('Irish.csv', skiprows=range(4),encoding='latin1')
columns = df.columns[1:]

finalData = {}

for i in range(len(columns)):
    finalScore = 0
    for j in range(len(columns)):
        if i != j:
            totalJ = 0
            # compare the ranks of each pairs and sum up the voters 
            totalJ = sum(np.where(df[columns[j]] >= df[columns[i]], df["number of voters"], 0))
            # store the highest voters             
            if( finalScore < totalJ):
                finalScore = totalJ
        # decide the KS(winner) for each candidate 
        if(j == len(columns) -1) :
            key = "KS("+columns[i]+")"
            finalData[key] = finalScore

sort_orders = sorted(finalData.items(), key=lambda x: x[1])

print("The order of the candidates elected in Kramer Simpson's rule are:")

for i in sort_orders:
    print(i[0] + " - ", i[1])

The order of the candidates elected in Kramer Simpson's rule are:
KS(Trevor Sargent G.P. ) -  26641.0
KS(Sean Ryan Lab ) -  29198.0
KS(Michael Kennedy F.F. ) -  31830.0
KS(Jim Glennon F.F. ) -  31933.0
KS(Clare Daly S.P. ) -  32716.0
KS(G.V. Wright F.F. ) -  33097.0
KS(Nora Owen F.G. ) -  34965.0
KS(Cathal Boland F.G. ) -  39576.0
KS(Mick Davis S.F. ) -  40403.0
KS(Ciaran Goulding Non-P ) -  40548.0
KS(Eamonn Quinn Non-P ) -  42519.0
KS(David Henry Walshe C.C. Csp ) -  43148.0


In [7]:
import pandas as pd
irishdf = pd.read_csv("Irish.csv",encoding = 'ISO-8859-1',skiprows=4)[:-1]

In [8]:
candidates = irishdf.columns[1:]

### The winner elected by the plurality rule

In [11]:
dis = {}
for can in candidates:
    candf = irishdf[['number of voters',can]]
    dis[can] = candf[candf[can] == 12]['number of voters'].sum()

dict(sorted(dis.items(), key=lambda item: item[1],reverse=True))

{'Trevor Sargent G.P. ': 7294.0,
 'Sean Ryan Lab ': 6359.0,
 'Jim Glennon F.F. ': 5892.0,
 'G.V. Wright F.F. ': 5658.0,
 'Clare Daly S.P. ': 5501.0,
 'Michael Kennedy F.F. ': 5253.0,
 'Nora Owen F.G. ': 4012.0,
 'Mick Davis S.F. ': 1350.0,
 'Cathal Boland F.G. ': 1177.0,
 'Ciaran Goulding Non-P ': 914.0,
 'Eamonn Quinn Non-P ': 285.0,
 'David Henry Walshe C.C. Csp ': 247.0}

### Copeland's preference order

In [19]:
Cope_score = {}
for can in candidates:
    Cope_score[can] = 0
for i in range(len(candidates)):
    for j in range(i+1,len(candidates)):
        cansdf= irishdf[['number of voters',candidates[i],candidates[j]]]
        cani = cansdf[cansdf[candidates[i]]>cansdf[candidates[j]]]['number of voters'].sum()
        canj = cansdf[cansdf[candidates[j]]>cansdf[candidates[i]]]['number of voters'].sum()
        print(candidates[i],'vs',candidates[j],':',cani,'to',canj)
        if cani > canj:
            Cope_score[candidates[i]] +=1
        elif cani < canj:
            Cope_score[candidates[j]] +=1
        else:
            Cope_score[candidates[i]] +=0.5
            Cope_score[candidates[j]] +=0.5

G.V. Wright F.F.  vs David Henry Walshe C.C. Csp  : 17114.0 to 1196.0
G.V. Wright F.F.  vs Trevor Sargent G.P.  : 14527.0 to 21205.0
G.V. Wright F.F.  vs Sean Ryan Lab  : 15142.0 to 18424.0
G.V. Wright F.F.  vs Eamonn Quinn Non-P  : 17144.0 to 2141.0
G.V. Wright F.F.  vs Nora Owen F.G.  : 16139.0 to 11876.0
G.V. Wright F.F.  vs Michael Kennedy F.F.  : 10845.0 to 12112.0
G.V. Wright F.F.  vs Ciaran Goulding Non-P  : 16971.0 to 5526.0
G.V. Wright F.F.  vs Jim Glennon F.F.  : 11609.0 to 12426.0
G.V. Wright F.F.  vs Mick Davis S.F.  : 16777.0 to 4555.0
G.V. Wright F.F.  vs Clare Daly S.P.  : 15685.0 to 14205.0
G.V. Wright F.F.  vs Cathal Boland F.G.  : 16822.0 to 7744.0
David Henry Walshe C.C. Csp  vs Trevor Sargent G.P.  : 794.0 to 24272.0
David Henry Walshe C.C. Csp  vs Sean Ryan Lab  : 1031.0 to 20772.0
David Henry Walshe C.C. Csp  vs Eamonn Quinn Non-P  : 1272.0 to 2317.0
David Henry Walshe C.C. Csp  vs Nora Owen F.G.  : 1179.0 to 13613.0
David Henry Walshe C.C. Csp  vs Michael Kennedy

In [32]:
print("The Coplants's preference order : ")

sorted_data = sorted(Cope_score.items(), key=lambda item: item[1],reverse=True)

for i in sorted_data:
    print(i[0] + " - ", i[1])

The Coplants's preference order : 
Trevor Sargent G.P.  -  11
Sean Ryan Lab  -  10
Michael Kennedy F.F.  -  9
Jim Glennon F.F.  -  8
G.V. Wright F.F.  -  7
Clare Daly S.P.  -  6
Nora Owen F.G.  -  5
Cathal Boland F.G.  -  4
Ciaran Goulding Non-P  -  3
Mick Davis S.F.  -  2
Eamonn Quinn Non-P  -  1
David Henry Walshe C.C. Csp  -  0
